In [ ]:
import re
import json
import numpy as np
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
import re
import numpy as np
import spacy
from spacy import displacy
import pandas as pd
import os
import logging
from tqdm import tqdm

In [ ]:
stop_words_path = '../data/weak_data/stop_words.txt'
stop_list = []
with open(stop_words_path, 'r') as f:
    for i in f.readlines():
        stop_list.append(i.strip()) 
    f.close()

In [ ]:
# def load_dict(file_path):
#     with open(file_path, 'r') as f:
#         dicts = json.load(f).decode('utf8')
#         f.close()
#     return dicts

# def Mesh_dict(mesh_path):
#   mesh_dict = {}
#   with open(mesh_path, encoding='utf-8') as file1:
#     mem_string = ""
#     for line in file1:
#       if line == '\n':
#         if mem_string.startswith('mesh:'):
#           med_list = mem_string.split(';')
#           try:
#             key = re.findall(r'mesh:[a-z]*[A-Z][0-9]+', med_list[0])[0]
#           except IndexError:
#             pass
#           mesh_dict[key] = {}
#           for i in med_list:
#             item = i.strip()
#             if item.startswith('skos:broader'):
#               mesh_dict[key]['broader'] = list()
#               mesh_dict[key]['broader'].extend(re.findall(r'[a-z]*:[a-z]*[A-Z]*[0-9]+', item))
#             if item.startswith('skos:prefLabel'):
#               # add other language
#               mesh_dict[key]['AllLabel'] = list()
#               if '"@fi' in item:
#                 mesh_dict[key]['AllLabel'].append(re.findall(r'".*"@fi', item)[0][1:-4])
#               if '"@en' in item:
#                 mesh_dict[key]['AllLabel'].append(re.findall(r'".*"@en', item)[0][1:-4])
#               if '"@sv' in item:
#                 mesh_dict[key]['AllLabel'].append(re.findall(r'".*"@sv', item)[0][1:-4])
#               try:
#                 mesh_dict[key]['prefLabel'] = re.findall(r'".*"@fi', item)[0][1:-4]
#               except IndexError:
#                 mesh_dict[key]['prefLabel'] = re.findall(r'".*"@en', item)[0][1:-4]
#             if item.startswith('skos:altLabel'):
#               mesh_dict[key]['altLabel'] = list()
#               alt_list = re.findall(r'".*"@fi', item)
#               for i in alt_list:
#                 mesh_dict[key]['altLabel'].append(i[1:-4])
#             if item.startswith('skos:narrower'):
#               mesh_dict[key]['narrower'] = list()
#               mesh_dict[key]['narrower'].extend(re.findall(r'[a-z]*:[a-z]*[A-Z]*[0-9]+', item))
#             if item.startswith('skos:related'):
#               mesh_dict[key]['related'] = list()
#               mesh_dict[key]['related'].extend(re.findall(r'[a-z]*:[a-z]*[A-Z]*[0-9]+', item))
#         mem_string = ""
#         key = ""
#       else:
#         mem_string = mem_string + line
        
#   return mesh_dict

# def Estab_rel(sum_dict):
#   rel_list = []
#   for k, v in sum_dict.items():
#     if 'broader' in list(v.keys()):
#       for b in v['broader']:
#         rel_list.append((b, k))
#     if 'narrower' in list(v.keys()):
#       for n in v['narrower']:
#         rel_list.append((k, n))
          
#   return rel_list

# def traverse(hierarchy, graph, names):
#     for name in names:
#         hierarchy[name] = traverse({}, graph, graph[name])
#     return hierarchy

# def Build_graph(rel_list):
#   graph = {name: set() for tup in rel_list for name in tup}
#   has_parent = {name: False for tup in rel_list for name in tup}
#   for parent, child in rel_list:
#       graph[parent].add(child)
#       has_parent[child] = True

#   roots = [name for name, parents in has_parent.items() if not parents]
  
#   res_dict = traverse({}, graph, roots)
  
#   return res_dict, roots

# def flatten(d):    
#   res = []  # Result list
#   for key, val in d.items():
#     res.append(key)
#     res += flatten(val)
#     if d == {}:
#       res = d
#   return res

# def Flat_dict(res_dict):
#   flat_dict = {}
#   for k, v in res_dict.items():
#     flat_dict[k] = flatten(v)
#   return flat_dict

# def Retrieve_tree(label, f_dicts, m_dicts, r_nodes):
#   label4code = [i[0] for i in r_nodes if i[1] == label][0]
#   item_list = f_dicts[label4code]
#   label_list = [m_dicts[j]['prefLabel'] for j in item_list]
#   return label_list

# # compressed dictionary
# def create_new_dicts(flat_dict, mesh_dict):
#   label2code = {}
#   for k, v in mesh_dict.items():
#     if k != '':
#       label2code[v['prefLabel']] = k
#   # comp_dicts = {
#   #   'Anatomical-structure': ['tuki- ja liikuntaelimistö', 'kudokset', 'hermosto', 'purentaelimistö', 'veri- ja immuunijärjestelmät', 'sydän ja verenkiertoelimistö', 'ruuansulatuselimistö', 'hengityselimet', 'umpieritysjärjestelmä', 'aistinelimet', 'virtsa- ja sukupuolielimet', 'kehonalueet', 'iho ja ihon apuelimet'],
#   #   'Body-function-and-measurement': ['lisääntymis- ja virtsaelinten fysiologiset ilmiöt', 'muskuloskeletaaliset ja hermostolliset fysiologiset ilmiöt', 'ruuansulatuskanavan ja suun fysiologiset ilmiöt', 'ihovaipan fysiologiset ilmiöt', 'fysiologiset ilmiöt', 'verenkierto- ja hengitysfysiologia', 'aineenvaihdunta'],
#   #   'Medical-condition': ['purentaelimistön sairaudet', 'patologiset tilat, löydökset ja oireet', 'hermoston sairaudet', 'kasvaimet', 'ravitsemus- ja aineenvaihduntasairaudet', 'korva-, nenä- ja kurkkutaudit', 'miesten virtsa- ja sukupuolielinten sairaudet', 'iho- ja sidekudossairaudet', 'tuki- ja liikuntaelinten sairaudet', 'immuunijärjestelmän sairaudet', 'endokriiniset sairaudet', 'hengityselinten taudit', 'veri- ja lymfaattiset taudit', 'naisen virtsa- ja sukupuolielinten sairaudet ja raskauskomplikaatiot', 'synnynnäiset, perinnölliset ja vastasyntyneen taudit ja epämuodostumat', 'sydän- ja verisuonitaudit', 'infektiot', 'silmätaudit', 'haavat ja vammat'],
#   #   'Medical-device': ['tarvikkeet ja varusteet'],
#   #   'Medical-procedure': ['leikkaukset', 'diagnoosi', 'hoitomenetelmät', 'informaatiotiede', 'anestesia ja analgesia', 'tutkimusmenetelmät'],
#   #   'Medication': ['lääkevalmisteet', 'polysykliset yhdisteet', 'kemikaalien vaikutukset ja käyttötarkoitukset', 'heterosykliset yhdisteet', 'epäorgaaniset kemikaalit', 'orgaaniset kemikaalit']
#   # }
#     comp_dicts = {
#     'Anatomical-structure': ['tuki- ja liikuntaelimistö', 'kudokset', 'hermosto', 'purentaelimistö', 'veri- ja immuunijärjestelmät', 'sydän ja verenkiertoelimistö', 'ruuansulatuselimistö', 'hengityselimet', 'umpieritysjärjestelmä', 'aistinelimet', 'virtsa- ja sukupuolielimet', 'kehonalueet', 'iho ja ihon apuelimet'],
#     'Body-function-and-measurement': ['lisääntymis- ja virtsaelinten fysiologiset ilmiöt', 'muskuloskeletaaliset ja hermostolliset fysiologiset ilmiöt', 'ruuansulatuskanavan ja suun fysiologiset ilmiöt', 'ihovaipan fysiologiset ilmiöt', 'fysiologiset ilmiöt', 'verenkierto- ja hengitysfysiologia', 'aineenvaihdunta'],
#     'Medical-condition': ['purentaelimistön sairaudet', 'patologiset tilat, löydökset ja oireet', 'hermoston sairaudet', 'kasvaimet', 'ravitsemus- ja aineenvaihduntasairaudet', 'korva-, nenä- ja kurkkutaudit', 'virtsa- ja sukupuolielinten sairaudet', 'iho- ja sidekudossairaudet', 'tuki- ja liikuntaelinten sairaudet', 'immuunijärjestelmän sairaudet', 'endokriiniset sairaudet', 'hengityselinten taudit', 'veri- ja lymfaattiset taudit', 'synnynnäiset, perinnölliset ja vastasyntyneen taudit ja epämuodostumat', 'sydän- ja verisuonitaudit', 'infektiot', 'silmätaudit', 'haavat ja vammat'],
#     'Medical-device': ['tarvikkeet ja varusteet'],
#     'Medical-procedure': ['leikkaukset', 'diagnoosi', 'hoitomenetelmät', 'informaatiotiede', 'anestesia ja analgesia', 'tutkimusmenetelmät'],
#     'Medication': ['lääkevalmisteet', 'polysykliset yhdisteet', 'kemikaalien vaikutukset ja käyttötarkoitukset', 'heterosykliset yhdisteet', 'epäorgaaniset kemikaalit', 'orgaaniset kemikaalit']
#   }
#   # then check each element in the medical dictionary
#   sub_item_collect = []
#   code_dicts = {}
#   for k, v in comp_dicts.items():
#     code_dicts[k] = []
#     for item in v:
#       code_dicts[k].append(label2code[item])
#     # append separate value list
#     sub_item_collect += v
#   # size of updated medical dictionary
#   size_dicts = len(sub_item_collect)
#   # use flatten dictionary to construct new dictionary
#   new_dicts = {}
#   for k, v in code_dicts.items():
#     new_dicts[k] = []
#     for item in v:
#       elements = flat_dict[item]
#       new_dicts[k] += elements
#       new_dicts[k] += [item]

#   return new_dicts

# def label_pooling(string):
#   str_set = []
#   str_element = string.split(';')
#   for i in str_element:
#     str_set.append(i.lower())
  
#   return str_set

# def token_mapping(source_tok, sub_dict):
#   matched_num = 0
#   matched_tok = []

#   if ' ' in source_tok:
#     source = source_tok.split(' ')
#   else:
#     source = [source_tok]
#   source_len = np.sum(np.array([len(i) for i in source]))
#   for item in sub_dict:
#     if ' ' in item:
#       piece = item.split(' ')
#     else:
#       piece = [item]
#     # num of terms is equal
#     if len(piece) == len(source):
#       matched_count = 0
#       for p in range(len(piece)):
#         # len of each piece >= len of source piece
#         if len(piece[p]) >= len(source[p]):
#           for t in range(len(source[p])):
#             if piece[p][t] == source[p][t]:
#               matched_count += 1
#             else:
#               break
#           if matched_count == source_len:
#             matched_num += 1
#             matched_tok.append(item)
#   return matched_num, matched_tok

# def candidates(source, dicts):
#   can_len = []
#   can = []
#   for k, v in dicts.items():
#     num, tok = token_mapping(source, v)
#     can_len.append(num)
#     can.append(tok)
#   return can_len, can

# def cal_score(source, cans):
#   scores = []
#   for cat in cans:
#     score = []
#     for item in cat:
#       score.append(fuzz.ratio(source, item))
#     if len(score) == 0:
#       scores.append(0)
#     else:
#       scores.append(np.max(np.array(score)))
#   return scores

# def label_assign(s1, s2):
#   s = []
#   for i in range(len(s1)):
#     if s1[i] > s2[i]:
#       s.append(s1[i])  
#     else:
#       s.append(s2[i])  
#   s = np.array(s)
#   if np.max(s) == 0:
#     max_idx = -1
#     max_value = 0
#   else:
#     max_value = 0
#     max_idx = 0
#     for z in range(len(s)):
#       if s[z] > max_value:
#         max_idx = z
#         max_value = s[z]
#       # if the values are equal, then comparison on two list
#       elif s[z] == max_value:
#         s1_z = s1[z]
#         s1_m = s1[max_idx]
#         s2_z = s2[z]
#         s2_m = s2[max_idx]
#         # one of list, the z win
#         if s1_z > s1_m and s2_z == s2_m:
#           max_idx = z
#         elif s1_z == s1_m and s2_z > s2_m:
#           max_idx = z
#         else:
#           pass

#   return max_idx, max_value

# def create_res_dicts(a_dict, m_dict):
#   res_dicts = {}
#   res_dicts_alt = {}
#   for k, v in a_dict.items():
#     md_items = v
#     label_lake = []
#     label_lake_alt = []
#     for i in md_items:
#         cur_item = m_dict[i]
#         if 'altLabel' in cur_item.keys():
#             cur_alt = cur_item['altLabel']
#         else:
#             cur_alt = []
#         cur_pref = cur_item['prefLabel']
#         cur_all = cur_item['AllLabel']
#         if len(cur_alt) > 0:
#             label_lake_alt += cur_alt
#         if cur_pref != '':
#           label_lake.append(cur_pref)
#         if len(cur_all) > 0:
#           label_lake += cur_all
#     label_lake = list(set(label_lake))
#     main_match_string = ';'.join(label_lake)
#     res_dicts[k] = label_pooling(main_match_string)

#     label_lake_alt = list(set(label_lake_alt))
#     main_match_string_alt = ';'.join(label_lake_alt)
#     res_dicts_alt[k] = label_pooling(main_match_string_alt)

#   return res_dicts, res_dicts_alt

# def beam_mapping(source, r_dicts, r_dicts_alt, window, mask_list):
  
#   labels = []
#   label_space =  list(r_dicts.keys())
#   source_ele = source#.split(' ')
#   source_len = len(source_ele)
#   idx = 0
  
#   while idx < source_len:
#     if mask_list[idx] == 'T':
#       if (source_len - idx) >= window:
#         step = window
#       else:
#         step = source_len - idx
#       candidate_idx = []
#       candidate_value = []
#       for shift in range(1, step+1):
#         term = source_ele[idx:idx + shift]
#         term_str = ' '.join(term)
#         _, candidate = candidates(term_str, r_dicts)
#         _, candidate_alt = candidates(term_str, r_dicts_alt)
#         s1 = cal_score(term_str, candidate)
#         s2 = cal_score(term_str, candidate_alt)
#         max_idx, max_value = label_assign(s1, s2)
#         candidate_idx.append(max_idx)
#         candidate_value.append(max_value)
#       if np.max(np.array(candidate_value)) > 0:
#         # the position of best matching
#         max_pos = np.argmax(np.array(candidate_value))
#         # the label index of best matching
#         label_idx = candidate_idx[max_pos]
#         win_shift = max_pos + 1
#         idx += win_shift
#         label = label_space[label_idx]
#         if win_shift == 1:
#           labels.append('B-' + label)
#         else:
#           labels.append('B-' + label)
#           for i in range(win_shift - 1):
#             labels.append('I-' + label)
#       else:
#         labels.append('O')
#         idx += 1
#     else:
#       labels.append('O')
#       idx += 1

#   return labels

# def lemma_tokens(input_path):
#   with open(input_path, 'r') as f:
#     out_str = f.readlines()
#     f.close()
#   origin = []
#   lemma = []
#   pos = []
#   idx = 0
#   while idx < len(out_str):
#     if out_str[idx].startswith("# text = "):
#       idx += 1
#       lemma_tmp = []
#       pos_tmp = []
#       orig_tmp = []
#       while out_str[idx] != '\n':
#         origi_i = out_str[idx].split('\t')[1].lower()
#         lemma_i = out_str[idx].split('\t')[2]
#         pos_tag = out_str[idx].split('\t')[3]
#         #print(lemma_i)
#         orig_tmp.append(origi_i.replace('#', ''))
#         lemma_tmp.append(lemma_i.replace('#', ''))
#         pos_tmp.append(pos_tag)
#         idx += 1
#       origin.append(orig_tmp)
#       lemma.append(lemma_tmp)
#       pos.append(pos_tmp)
#     else:
#       idx += 1
#   mask = []
#   for s_idx in range(len(pos)):
#     sent_pos = pos[s_idx]
#     sent = lemma[s_idx]
#     mask_tmp = []
#     for tok_idx in range(len(sent)):
#       if sent_pos[tok_idx] == 'NOUN':
#         if sent[tok_idx] not in ['mm', 'cm', 'm', 'g', 'kg']:
#           mask_tmp.append('T')
#         else:
#           mask_tmp.append('F')
#       else:
#         mask_tmp.append('F')
#       # mask_tmp.append('T')
#     mask.append(mask_tmp)
    
#   return origin, lemma, mask

# def write_list(file_path, data_list):
#   res_str = ''
#   for item in data_list:
#     res_str += ';'.join(item)
#     res_str += '\n'
#   with open(file_path, 'w+') as f:
#     f.write(res_str)
#     f.close()

In [ ]:
def load_dict(file_path):
    with open(file_path, 'r') as f:
        dicts = json.load(f).decode('utf8')
        f.close()
    return dicts

def Mesh_dict(mesh_path):
  mesh_dict = {}
  with open(mesh_path, encoding='utf-8') as file1:
    mem_string = ""
    for line in file1:
      if line == '\n':
        if mem_string.startswith('mesh:'):
          med_list = mem_string.split(';')
          try:
            key = re.findall(r'mesh:[a-z]*[A-Z][0-9]+', med_list[0])[0]
          except IndexError:
            pass
          mesh_dict[key] = {}
          for i in med_list:
            item = i.strip()
            if item.startswith('skos:broader'):
              mesh_dict[key]['broader'] = list()
              mesh_dict[key]['broader'].extend(re.findall(r'[a-z]*:[a-z]*[A-Z]*[0-9]+', item))
            if item.startswith('skos:prefLabel'):
              # add other language
              mesh_dict[key]['AllLabel'] = list()
              if '"@fi' in item:
                mesh_dict[key]['AllLabel'].append(re.findall(r'".*"@fi', item)[0][1:-4])
              if '"@en' in item:
                mesh_dict[key]['AllLabel'].append(re.findall(r'".*"@en', item)[0][1:-4])
              if '"@sv' in item:
                mesh_dict[key]['AllLabel'].append(re.findall(r'".*"@sv', item)[0][1:-4])
              try:
                mesh_dict[key]['prefLabel'] = re.findall(r'".*"@fi', item)[0][1:-4]
              except IndexError:
                mesh_dict[key]['prefLabel'] = re.findall(r'".*"@en', item)[0][1:-4]
            if item.startswith('skos:altLabel'):
              mesh_dict[key]['altLabel'] = list()
              alt_list = re.findall(r'".*"@fi', item)
              for i in alt_list:
                mesh_dict[key]['altLabel'].append(i[1:-4])
            if item.startswith('skos:narrower'):
              mesh_dict[key]['narrower'] = list()
              mesh_dict[key]['narrower'].extend(re.findall(r'[a-z]*:[a-z]*[A-Z]*[0-9]+', item))
            if item.startswith('skos:related'):
              mesh_dict[key]['related'] = list()
              mesh_dict[key]['related'].extend(re.findall(r'[a-z]*:[a-z]*[A-Z]*[0-9]+', item))
        mem_string = ""
        key = ""
      else:
        mem_string = mem_string + line
        
  return mesh_dict

def Estab_rel(sum_dict):
  rel_list = []
  for k, v in sum_dict.items():
    if 'broader' in list(v.keys()):
      for b in v['broader']:
        rel_list.append((b, k))
    if 'narrower' in list(v.keys()):
      for n in v['narrower']:
        rel_list.append((k, n))
          
  return rel_list

def traverse(hierarchy, graph, names):
    for name in names:
        hierarchy[name] = traverse({}, graph, graph[name])
    return hierarchy

def Build_graph(rel_list):
  graph = {name: set() for tup in rel_list for name in tup}
  has_parent = {name: False for tup in rel_list for name in tup}
  for parent, child in rel_list:
      graph[parent].add(child)
      has_parent[child] = True

  roots = [name for name, parents in has_parent.items() if not parents]
  
  res_dict = traverse({}, graph, roots)
  
  return res_dict, roots

def flatten(d):    
  res = []  # Result list
  for key, val in d.items():
    res.append(key)
    res += flatten(val)
    if d == {}:
      res = d
  return res

def Flat_dict(res_dict):
  flat_dict = {}
  for k, v in res_dict.items():
    flat_dict[k] = flatten(v)
  return flat_dict

def Retrieve_tree(label, f_dicts, m_dicts, r_nodes):
  label4code = [i[0] for i in r_nodes if i[1] == label][0]
  item_list = f_dicts[label4code]
  label_list = [m_dicts[j]['prefLabel'] for j in item_list]
  return label_list

# compressed dictionary
def create_new_dicts(flat_dict, mesh_dict):
  label2code = {}
  for k, v in mesh_dict.items():
    if k != '':
      label2code[v['prefLabel']] = k
  # comp_dicts = {
  #   'Anatomical-structure': ['tuki- ja liikuntaelimistö', 'kudokset', 'hermosto', 'purentaelimistö', 'veri- ja immuunijärjestelmät', 'sydän ja verenkiertoelimistö', 'ruuansulatuselimistö', 'hengityselimet', 'umpieritysjärjestelmä', 'aistinelimet', 'virtsa- ja sukupuolielimet', 'kehonalueet', 'iho ja ihon apuelimet'],
  #   'Body-function-and-measurement': ['lisääntymis- ja virtsaelinten fysiologiset ilmiöt', 'muskuloskeletaaliset ja hermostolliset fysiologiset ilmiöt', 'ruuansulatuskanavan ja suun fysiologiset ilmiöt', 'ihovaipan fysiologiset ilmiöt', 'fysiologiset ilmiöt', 'verenkierto- ja hengitysfysiologia', 'aineenvaihdunta'],
  #   'Medical-condition': ['purentaelimistön sairaudet', 'patologiset tilat, löydökset ja oireet', 'hermoston sairaudet', 'kasvaimet', 'ravitsemus- ja aineenvaihduntasairaudet', 'korva-, nenä- ja kurkkutaudit', 'miesten virtsa- ja sukupuolielinten sairaudet', 'iho- ja sidekudossairaudet', 'tuki- ja liikuntaelinten sairaudet', 'immuunijärjestelmän sairaudet', 'endokriiniset sairaudet', 'hengityselinten taudit', 'veri- ja lymfaattiset taudit', 'naisen virtsa- ja sukupuolielinten sairaudet ja raskauskomplikaatiot', 'synnynnäiset, perinnölliset ja vastasyntyneen taudit ja epämuodostumat', 'sydän- ja verisuonitaudit', 'infektiot', 'silmätaudit', 'haavat ja vammat'],
  #   'Medical-device': ['tarvikkeet ja varusteet'],
  #   'Medical-procedure': ['leikkaukset', 'diagnoosi', 'hoitomenetelmät', 'informaatiotiede', 'anestesia ja analgesia', 'tutkimusmenetelmät'],
  #   'Medication': ['lääkevalmisteet', 'polysykliset yhdisteet', 'kemikaalien vaikutukset ja käyttötarkoitukset', 'heterosykliset yhdisteet', 'epäorgaaniset kemikaalit', 'orgaaniset kemikaalit']
  # }
    # 'Body-function-and-measurement': ['terveydenhuollon ammatit', 'ihmisten toiminnot', 'informaatiotiede', 
    # 'Medical-procedure': ['henkilöt',
    comp_dicts = {
    'Anatomical-structure': ['tuki- ja liikuntaelimistö', 'kudokset', 'hermosto', 'purentaelimistö', 'veri- ja immuunijärjestelmät', 'sydän ja verenkiertoelimistö', 'ruuansulatuselimistö', 'hengityselimet', 'umpieritysjärjestelmä', 'aistinelimet', 'virtsa- ja sukupuolielimet', 'kehonalueet', 'iho ja ihon apuelimet'],
    'Body-function-and-measurement': ['lisääntymis- ja virtsaelinten fysiologiset ilmiöt', 'muskuloskeletaaliset ja hermostolliset fysiologiset ilmiöt', 'ruuansulatuskanavan ja suun fysiologiset ilmiöt', 'ihovaipan fysiologiset ilmiöt', 'fysiologiset ilmiöt', 'verenkierto- ja hengitysfysiologia', 'aineenvaihdunta'],
    'Medical-condition': ['purentaelimistön sairaudet', 'patologiset tilat, löydökset ja oireet', 'hermoston sairaudet', 'kasvaimet', 'ravitsemus- ja aineenvaihduntasairaudet', 'korva-, nenä- ja kurkkutaudit', 'virtsa- ja sukupuolielinten sairaudet', 'iho- ja sidekudossairaudet', 'tuki- ja liikuntaelinten sairaudet', 'immuunijärjestelmän sairaudet', 'endokriiniset sairaudet', 'hengityselinten taudit', 'veri- ja lymfaattiset taudit', 'synnynnäiset, perinnölliset ja vastasyntyneen taudit ja epämuodostumat', 'sydän- ja verisuonitaudit', 'infektiot', 'silmätaudit', 'haavat ja vammat'],
    'Medical-device': ['tarvikkeet ja varusteet'],
    'Medical-procedure': ['leikkaukset', 'diagnoosi', 'hoitomenetelmät', 'anestesia ja analgesia', 'tutkimusmenetelmät'],
    'Medication': ['henkilöt','lääkevalmisteet', 'polysykliset yhdisteet', 'kemikaalien vaikutukset ja käyttötarkoitukset', 'heterosykliset yhdisteet', 'epäorgaaniset kemikaalit', 'orgaaniset kemikaalit']
  }
  # then check each element in the medical dictionary
  sub_item_collect = []
  code_dicts = {}
  for k, v in comp_dicts.items():
    # print(v)
    code_dicts[k] = []
    for item in v:
      code_dicts[k].append(label2code[item])
    # append separate value list
    sub_item_collect += v
  # size of updated medical dictionary
  size_dicts = len(sub_item_collect)
  # use flatten dictionary to construct new dictionary
  new_dicts = {}
  for k, v in code_dicts.items():
    new_dicts[k] = []
    for item in v:
      elements = flat_dict[item]
      new_dicts[k] += elements
      new_dicts[k] += [item]

  return new_dicts

def label_pooling(string):
  str_set = []
  str_element = string.split(';')
  for i in str_element:
    str_set.append(i.lower())
  
  return str_set

def token_mapping(source_tok, sub_dict):
  matched_num = 0
  matched_tok = []

  if ' ' in source_tok:
    source = source_tok.split(' ')
  else:
    source = [source_tok]
  source_len = np.sum(np.array([len(i) for i in source]))
  for item in sub_dict:
    if ' ' in item:
      piece = item.split(' ')
    else:
      piece = [item]
    # num of terms is equal
    if len(piece) == len(source):
      matched_count = 0
      for p in range(len(piece)):
        # len of each piece >= len of source piece
        if len(piece[p]) >= len(source[p]):
          for t in range(len(source[p])):
            if piece[p][t] == source[p][t]:
              matched_count += 1
            else:
              break
          if matched_count == source_len:
            matched_num += 1
            matched_tok.append(item)
  return matched_num, matched_tok

def candidates(source, dicts):
  can_len = []
  can = []
  for k, v in dicts.items():
    num, tok = token_mapping(source, v)
    can_len.append(num)
    can.append(tok)
  return can_len, can

def cal_score(source, cans):
  scores = []
  for cat in cans:
    score = []
    for item in cat:
      score.append(fuzz.ratio(source, item))
    if len(score) == 0:
      scores.append(0)
    else:
      scores.append(np.max(np.array(score)))
  return scores

def label_assign(s1, s2):
  s = []
  for i in range(len(s1)):
    if s1[i] > s2[i]:
      s.append(s1[i])  
    else:
      s.append(s2[i])  
  s = np.array(s)
  if np.max(s) == 0:
    max_idx = -1
    max_value = 0
  else:
    max_value = 0
    max_idx = 0
    for z in range(len(s)):
      if s[z] > max_value:
        max_idx = z
        max_value = s[z]
      # if the values are equal, then comparison on two list
      elif s[z] == max_value:
        s1_z = s1[z]
        s1_m = s1[max_idx]
        s2_z = s2[z]
        s2_m = s2[max_idx]
        # one of list, the z win
        if s1_z > s1_m and s2_z == s2_m:
          max_idx = z
        elif s1_z == s1_m and s2_z > s2_m:
          max_idx = z
        else:
          pass

  return max_idx, max_value

def create_res_dicts(a_dict, m_dict):
  res_dicts = {}
  res_dicts_alt = {}
  for k, v in a_dict.items():
    md_items = v
    label_lake = []
    label_lake_alt = []
    for i in md_items:
        cur_item = m_dict[i]
        if 'altLabel' in cur_item.keys():
            cur_alt = cur_item['altLabel']
        else:
            cur_alt = []
        cur_pref = cur_item['prefLabel']
        cur_all = cur_item['AllLabel']
        if len(cur_alt) > 0:
            label_lake_alt += cur_alt
        if cur_pref != '':
          label_lake.append(cur_pref)
        if len(cur_all) > 0:
          label_lake += cur_all
    label_lake = list(set(label_lake))
    main_match_string = ';'.join(label_lake)
    res_dicts[k] = label_pooling(main_match_string)

    label_lake_alt = list(set(label_lake_alt))
    main_match_string_alt = ';'.join(label_lake_alt)
    res_dicts_alt[k] = label_pooling(main_match_string_alt)

  return res_dicts, res_dicts_alt



def lemma_tokens(input_path):
  with open(input_path, 'r') as f:
    out_str = f.readlines()
    f.close()
  origin = []
  lemma = []
  pos = []
  case = []
  idx = 0
  while idx < len(out_str):
    if out_str[idx].startswith("# text = "):
      idx += 1
      lemma_tmp = []
      pos_tmp = []
      orig_tmp = []
      case_tmp = []
      while out_str[idx] != '\n':
        trg_str = out_str[idx].split('\t')
        # raise ValueError('1')
        origi_i = trg_str[1].lower()
        lemma_i = trg_str[2]
        pos_tag = trg_str[3]
        
        #print(lemma_i)
        orig_tmp.append(origi_i.replace('#', ''))
        lemma_tmp.append(lemma_i)
        pos_tmp.append(pos_tag)
        case_tmp.append(trg_str[5])
        idx += 1
      origin.append(orig_tmp)
      lemma.append(lemma_tmp)
      pos.append(pos_tmp)
      case.append(case_tmp)
    else:
      idx += 1
  mask = []
  for s_idx in range(len(pos)):
    sent_pos = pos[s_idx]
    sent = lemma[s_idx]
    sent_case = case[s_idx]
    mask_tmp = []
    for tok_idx in range(len(sent)):
      if sent_pos[tok_idx] == 'NOUN':
        # print(sent[tok_idx])
        # print(len(sent[tok_idx]))
        # print(sent[tok_idx].isalpha())
        # print(sent[tok_idx] not in stop_list)
        # raise ValueError('1')
        # print(sent[tok_idx])
        if sent[tok_idx] not in stop_list and sent[tok_idx].replace('#', '').isalpha() and len(sent[tok_idx]) > 2:
          # print(sent[tok_idx])
          # print(sent[tok_idx].isalpha())
          # raise ValueError('1')
          mask_tmp.append('T')
        # elif '_' in sent_case[tok_idx]:# or 'Abbr=Yes' in sent_case[tok_idx]:
        #   mask_tmp.append('F')
        else:
          mask_tmp.append('F')
      else:
        mask_tmp.append('F')
      # mask_tmp.append('T')
    mask.append(mask_tmp)
    
  return origin, lemma, mask, case, pos

def write_list(file_path, data_list):
  res_str = ''
  for item in data_list:
    res_str += ';'.join(item)
    res_str += '\n'
  with open(file_path, 'w+') as f:
    f.write(res_str)
    f.close()
    

# def expand_item(dict_1, dict_2):
#     new_res_list = []
#     for m in dict_1:
#         #non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()).split() if item != '']
#         non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()) if item != '']
#         new_res_list.append(' '.join(non_alpha_list))
#         new_res_list += non_alpha_list
    
#     for m in dict_2:
#         #non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()).split() if item != '']
#         non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()) if item != '']
#         new_res_list.append(' '.join(non_alpha_list))
#         new_res_list += non_alpha_list
        
#     return list(set(new_res_list))

def expand_item(dict_1, dict_2):
    alpha_dict_1 = {}
    alpha_dict_2 = {}
    alpha_dict_3 = {}
    # chara_ = []
    for m in dict_1:
        #print(m)
        #print(re.sub(r'[^a-z]', ' ', m.lower()))
        non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()).split() if item != '']
        # alpha_dict_1 = {}
        # alpha_dict_2 = {}
        # alpha_dict_3 = {}
        # non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()) if item != '']
        # print(non_alpha_list)
        # raise ValueError('1')
        if len(non_alpha_list) == 1:
            #print(non_alpha_list)
            #raise ValueError('1')
            # chara_.append(non_alpha_list[0][0])
            tok_ = ' '.join(non_alpha_list)
            if tok_[0] not in list(alpha_dict_1.keys()):
                alpha_dict_1[tok_[0]] = set()
            alpha_dict_1[tok_[0]].add(tok_)
        elif len(non_alpha_list) == 2:
            # new_res_list_2.append(' '.join(non_alpha_list))
            tok_ = ' '.join(non_alpha_list)
            if tok_[0] not in list(alpha_dict_2.keys()):
                alpha_dict_2[tok_[0]] = set()
            alpha_dict_2[tok_[0]].add(tok_)
        else:
            # new_res_list_3.append(' '.join(non_alpha_list))
            tok_ = ' '.join(non_alpha_list)
            if tok_[0] not in list(alpha_dict_3.keys()):
                alpha_dict_3[tok_[0]] = set()
            alpha_dict_3[tok_[0]].add(tok_)
        # new_res_list += non_alpha_list
    
    for m in dict_2:
        #print(m)
        #print(re.sub(r'[^a-z]', ' ', m.lower()))
        non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()).split() if item != '']
        # alpha_dict_1 = {}
        # alpha_dict_2 = {}
        # alpha_dict_3 = {}
        #non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()) if item != '']
        # print(non_alpha_list)
        # raise ValueError('1')
        if len(non_alpha_list) == 1:
            #print(non_alpha_list)
            #raise ValueError('1')
            # chara_.append(non_alpha_list[0][0])
            tok_ = ' '.join(non_alpha_list)
            if tok_[0] not in list(alpha_dict_1.keys()):
                alpha_dict_1[tok_[0]] = set()
            alpha_dict_1[tok_[0]].add(tok_)
        elif len(non_alpha_list) == 2:
            # new_res_list_2.append(' '.join(non_alpha_list))
            tok_ = ' '.join(non_alpha_list)
            if tok_[0] not in list(alpha_dict_2.keys()):
                alpha_dict_2[tok_[0]] = set()
            alpha_dict_2[tok_[0]].add(tok_)
        else:
            # print(non_alpha_list)
            # raise ValueError('1')
            # new_res_list_3.append(' '.join(non_alpha_list))
            tok_ = ' '.join(non_alpha_list)
            if tok_[0] not in list(alpha_dict_3.keys()):
                alpha_dict_3[tok_[0]] = set()
            alpha_dict_3[tok_[0]].add(tok_)
        # new_res_list += non_alpha_list
    # print(set(chara_))
    #raise ValueError('1')
    return [alpha_dict_1, alpha_dict_2, alpha_dict_3]

def expand_item_pool(dict_1, dict_2):
    alpha_dict_1 = {}
    # alpha_dict_2 = {}
    # alpha_dict_3 = {}
    # chara_ = []
    for m in dict_1:
        non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()).split() if item != '']
        # tok_ = ' '.join(non_alpha_list)
        for tok_ in non_alpha_list:
            if tok_[0] not in list(alpha_dict_1.keys()):
                alpha_dict_1[tok_[0]] = set()
            if len(tok_) >= 2:
                alpha_dict_1[tok_[0]].add(tok_)
    
    for m in dict_2:
        #print(m)
        #print(re.sub(r'[^a-z]', ' ', m.lower()))
        non_alpha_list = [item for item in re.sub(r'[^a-z]', ' ', m.lower()).split() if item != '']
        # tok_ = ' '.join(non_alpha_list)
        for tok_ in non_alpha_list:
            if tok_[0] not in list(alpha_dict_1.keys()):
                alpha_dict_1[tok_[0]] = set()
            if len(tok_) >= 2:
                alpha_dict_1[tok_[0]].add(tok_)
    # print(set(chara_))
    #raise ValueError('1')
    return alpha_dict_1

def reverse_dict():
    # reverse dict

    name_list = ['Anatomical-structure', 'Body-function-and-measurement', 'Medical-condition', 'Medical-device', 'Medical-procedure', 'Medication']
    # sets = 'dict_alt_'
    sets = 'dict_'

    dict_ = {}
    for name in name_list:
        file_name_1 = sets + name + '.txt'
        read_path = '../data/dict/post/processed/{}'.format(file_name_1)
        res_list = []
        with open(read_path, 'r') as f:
            for i in f.readlines():
                res_list.append(i.strip())
            f.close()
        dict_[name] = res_list

    # sets = 'dict_alt_'

    dict_alt_ = {}
    sets = 'dict_alt_'

    for name in name_list:
        file_name_1 = sets + name + '.txt'
        read_path = '../data/dict/post/processed/{}'.format(file_name_1)
        res_list = []
        with open(read_path, 'r') as f:
            for i in f.readlines():
                res_list.append(i.strip())
            f.close()
        dict_alt_[name] = res_list
    
    return dict_, dict_alt_

def beam_mapping(source_ele, r_dicts, r_dicts_alt, window, mask_list, check_thres):
  
  # print(source)
  labels = []
  # label_space =  list(r_dicts.keys())
  ratio_thres = 0.5
  # if ratio_score is too low --> subwording --> lose too much information
  
  # source_ele = source#.split(' ')
  source_len = len(source_ele)
  idx = 0
    
  new_source_ele = []
  for source in source_ele:
        # print(source)
        if '#' in source:
            source_list = source.split('#')
            # print(source_list)
            ratio_score = len(source_list[0])/len(''.join(source_list))
            # print(ratio_score)
            if ratio_score < ratio_thres:
                source = ''.join(source_list)
            else:
                source = source_list[0]
            # print(ratio_score)
            # print(source)
        new_source_ele.append(source)
  source_ele = new_source_ele
    # raise ValueError(1)
    
  name_list = ['Anatomical-structure', 'Body-function-and-measurement', 'Medical-condition', 'Medical-device', 'Medical-procedure', 'Medication']
        
  res_dict = {}

  for i in name_list:
    res = expand_item(r_dicts[i], r_dicts_alt[i])
    # res = expand_item_pool(r_dicts[i], r_dicts_alt[i])
    # print(res[0])
    # raise ValueError('1')
    # print(len(res[1]))
    # print(len(res[2]))
    # print('-'*10)
    res_dict[i] = res

  # raise ValueError('1')
  mask_list[0]
  while idx < source_len:
    if mask_list[idx] == 'T':
      if (source_len - idx) >= window:
        step = window
      else:
        step = source_len - idx
      candidate_idx = []
      candidate_value = []
      for shift in range(1, step+1):
        term = source_ele[idx:idx + shift]
        term_str = ' '.join(term)
        # !!! original !!!
        # _, candidate = candidates(term_str, r_dicts)
        # _, candidate_alt = candidates(term_str, r_dicts_alt)
        # s1 = cal_score(term_str, candidate)
        # s2 = cal_score(term_str, candidate_alt)
        # max_idx, max_value = label_assign(s1, s2)
        # !!! no accelerating !!!
        # source = term_str
        # score_list = []
        # for i in name_list:
        #     res = expand_item(r_dicts[i], r_dicts_alt[i])
        #     max_val = 0
        #     # max_idx = 0
        #     for j in range(len(res)):
        #         item = res[j]
        #         if fuzz.ratio(source, item) > max_val:
        #             max_val = fuzz.ratio(source, item)
        #             # max_idx = j
        #     score_list.append(max_val)
        # !!! accelerating
        source = term_str
        score_list = []

        for k, v in res_dict.items():
            max_val = 0
            # max_idx = 0
            # print(v)
            # raise ValueError(1)
            # fir_tok = source[0]
            # print(v[0])
            
            # if fir_tok in v[0]:
            # -----------------------
            # code below --> map tokens in phrase
            if len(source.split()) == 1:
                if source[0] in list(v[0].keys()):
                    v_list = v[0][source[0]]
                else:
                    v_list = set()
            elif len(source.split()) == 2:
                if source[0] in list(v[1].keys()):
                    v_list = v[1][source[0]]
                else:
                    v_list = set()
            elif len(source.split()) == 3:
                if source[0] in list(v[2].keys()):
                    v_list = v[2][source[0]]
                    # print(v_list)
                    # raise ValueError(1)
                else:
                    v_list = set()
            v_list_ = list(v_list)
            # -----------------------
            # code below --> pooling all phrases
            # v_list_ = 
            # for j in range(len(v_list_)):
            #     item = v_list_[j]
            #     item_list = item.split()
#             source_list = source.split()
#             # print(source)
#             # map 1 token; 2 token; 3 token
#             sum_max_val = 0
#             for z in range(len(source_list)):
#                 source_tok = source_list[z]
#                 if source_tok[0] in list(v.keys()):
#                     item_list = v[source_tok[0]]
#                 else:
#                     item_list = []
#                 # print(item_list)
#                 # raise ValueError(1)
#                 tmp_max_val = 0
#                 for item_tok in item_list:
#                     # print(source_list)
#                     # print(source_tok)
#                     # print(item_list)
#                     # raise ValueError(1)
#                     # for zz in 
#                     map_len = int(len(source_tok)*check_thres)
#                     # print(item)
#                     if len(item_tok) < map_len:
#                         break
#                     else:
#                         # if fuzz.ratio(source, item) > max_val:
#                         #     max_val = fuzz.ratio(source, item)
#                         if source_tok[:map_len] == item_tok[:map_len]:
#                             # print(source[:map_len])
#                             # print(item)
#                             # print(source_tok)
#                             # print(item_tok)
#                             # raise ValueError(1)
#                             IoU_score = map_len/len(item_tok)
#                             # print(IoU_score)
#                             # raise ValueError(1)
#                             if IoU_score > sum_max_val:
#                                 tmp_max_val = IoU_score
#                 sum_max_val += tmp_max_val
#             sum_max_val = sum_max_val#/len(source_list)
#             if sum_max_val > max_val:
#                 max_val = sum_max_val
                
#                 # fuzz_score = fuzz.ratio(source, item)
#                 # # print(source)
#                 # # print(item)
#                 # # raise ValueError('1')
#                 # if fuzz_score > max_val:
#                 #     max_val = fuzz_score
#                 #     # max_idx = j
#             # score_list.append(max_val/len(source_list))
#             score_list.append(max_val)
            # -------------------------
        # print(source)
        # print('_'*10)
        # print(score_list)
        # raise ValueError(1)
            # map_thres = 0.9
            # print(source)
            # map_len = int(len(source)*check_thres)
            # print(map_len)
            # print(source[:map_len])
            # print(len(source.split()))
            # raise ValueError(1)
            for j in range(len(v_list_)):
                item = v_list_[j]
                item_list = item.split()
                source_list = source.split()
                # map 1 token; 2 token; 3 token
                sum_max_val = 0
                # print(source_list)
                # print(item_list)
                # raise ValueError(1)
                for z in range(len(source_list)):
                    source_tok = source_list[z]
                    item_tok = item_list[z]
                    map_len = int(len(source_tok)*check_thres)
                    # print(item)
                    if len(item_tok) < map_len:
                        break
                    else:
                        # if fuzz.ratio(source, item) > max_val:
                        #     max_val = fuzz.ratio(source, item)
                        # raise ValueError(1)
                        if source_tok[:map_len] == item_tok[:map_len]:
                            # print(source[:map_len])
                            # print(item)
                            IoU_score = map_len/len(item)
                            if IoU_score > sum_max_val:
                                sum_max_val = IoU_score
                                
                if sum_max_val > max_val:
                    max_val = sum_max_val
                
                # fuzz_score = fuzz.ratio(source, item)
                # # print(source)
                # # print(item)
                # # raise ValueError('1')
                # if fuzz_score > max_val:
                #     max_val = fuzz_score
                #     # max_idx = j
            score_list.append(max_val/len(source_list))
        # raise ValueError(1)
        # print(source)
        # print(source)
        # print(score_list)
        # print(term_str)
        # score_list = []
        # for i in label_space:
        #     res = expand_item(r_dicts[i], r_dicts_alt[i])
        #     max_val = 0
        #     # max_idx = 0
        #     for j in range(len(res)):
        #         item = res[j]
        #         if fuzz.ratio(term_str, item) > max_val:
        #             max_val = fuzz.ratio(source, item)
        #             # max_idx = j
        #     score_list.append(max_val)
        # print(score_list)
        # max_score = np.max(score_list)
        candidate_idx.append(np.argmax(score_list))
        candidate_value.append(np.max(score_list))
        
      # raise ValueError(1)
      if np.max(np.array(candidate_value)) > 0.5:
        # the position of best matching
        max_pos = np.argmax(np.array(candidate_value))
        # the label index of best matching
        label_idx = candidate_idx[max_pos]
        win_shift = max_pos + 1
        idx += win_shift
        label = name_list[label_idx]
        if win_shift == 1:
          labels.append('B-' + label)
        else:
          labels.append('B-' + label)
          for i in range(win_shift - 1):
            labels.append('I-' + label)
      else:
        labels.append('O')
        idx += 1
    else:
      labels.append('O')
      idx += 1

  return labels


In [ ]:
if __name__ == '__main__':
    # obtain labeling dicts
    sets = 'test'
    input_path = '../data/lemma_data/{}/1.conllu'.format(sets)
    mesh_path = '../tools/fin-mesh/mesh-skos.ttl'
    sent_path = '../data/processed_data/{}/1.txt'.format(sets)
    label_path = '../data/processed_data/{}/label.txt'.format(sets)

    mesh_dict = Mesh_dict(mesh_path)
    rel_list = Estab_rel(mesh_dict)
    window = 3
    # mesh:D055754 is the top one, also it is a English term -> remove
    res_dict, _ = Build_graph(rel_list)
    flat_dict = Flat_dict(res_dict)
    anno_dict = create_new_dicts(flat_dict, mesh_dict)
    # create res dict with label lakes
    # res_dicts, res_dicts_alt = create_res_dicts(anno_dict, mesh_dict)
    res_dicts, res_dicts_alt = reverse_dict()
    origin, lemma, mask, case, pos = lemma_tokens(input_path)
    # origin, lemma, mask = lemma_tokens(input_path)
    
    # raise ValueError('1')

    labels_list = []
    token_list = []
    # print(len(origin))
    for s_idx in tqdm(range(len(origin))):
        #origin_tmp = ' '.join(origin[s_idx])
        #lemma_tmp = ' '.join(lemma[s_idx])
        # s_idx = 1
        # labels_tmp_list = []
        # for ix in range(50):
        #     lemma[s_idx] = [item[ix] for item in res_list] + ['.']
        #     labels_tmp = beam_mapping(lemma[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx], 1.0)
        # # labels_tmp = beam_mapping(origin[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx])
        #     # print(labels_tmp)
        #     labels_tmp_list.append(labels_tmp)
        labels_tmp = beam_mapping(lemma[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx], 1.0)
        print(labels_tmp)
        raise ValueError(1)
        # print(' '.join(labels_tmp))
        # collection
        labels_list.append(labels_tmp)
        token_list.append(origin[s_idx])
        # if s_idx > 10000:
        #     break
    # write to the file
    write_list(label_path, labels_list)
    write_list(sent_path, token_list)
    # print('Done')
  

In [ ]:
label = ['Anatomical-structure', 'Body-function-and-measurement', 'Medical-condition', 'Medical-device', 'Medical-procedure', 'Medication']
bio_label = []
for i in label:
    bio_label.append('B-' + i)
    bio_label.append('I-' + i)
bio_label.append('O')

In [ ]:
idx_2_label = {}
for idx in range(len(bio_label)):
    idx_2_label[idx] = bio_label[idx]

In [ ]:
label_2_idx = {v:k for k,v in idx_2_label.items()}

In [ ]:
mx = np.zeros([len(labels_tmp_list[0]), len(idx_2_label)])
for i in range(len(labels_tmp_list)):
    item = labels_tmp_list[i]
    for j in range(len(item)):
        if item[j] != 'O':
            mx[j][label_2_idx[item[j]]] += 1

In [ ]:
label_2_idx

In [ ]:
mx

In [ ]:
label_2_idx

In [ ]:
len(labels_tmp_list[0])

In [ ]:
len(idx_2_label)

In [ ]:
j

In [ ]:
mx.shape

In [ ]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import torch

In [ ]:
tokenizer_path = '../tools/bert-base-finnish-uncased-v1'
mlm_model_path = '../save_models/pretrain_weight/checkpoint-100000'

In [ ]:
def predict_masked_sent(text, top_k=5):
    # Tokenize input
    res_topk = []
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        # print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))
        # print(predicted_token)
        # choose the best token
        res_topk.append(predicted_token)
    return res_topk

In [ ]:
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
model = BertForMaskedLM.from_pretrained(mlm_model_path)
model.eval()
# predict_masked_sent("My [MASK] is so cute.", top_k=5)

In [ ]:
input_str = ""
input_list = input_str.split()[:-1]
res_list = []
for i in range(len(input_list)):
    mask_tok = input_list[i]
    tmp_str = ' '.join(input_list) + '.'
    res_str = tmp_str.replace(mask_tok, '[MASK]')
    res_list.append(predict_masked_sent(res_str, top_k=20))
res_list
    # print('-'*10)
# predict_masked_sent(, top_k=5)


In [ ]:
len(res_list)

In [ ]:
len(mask[1])

In [ ]:
origin[0]

In [ ]:
lemma[0]

In [ ]:
[item[0] for item in res_list] + ['.']

In [ ]:
# res_dicts, res_dicts_alt = create_res_dicts(anno_dict, mesh_dict)

# # trg_dict = res_dicts_alt
# # sets = 'dict_alt_'
# # trg_dict = res_dicts
# # sets = 'dict_'

# for sets in ['dict_alt_', 'dict_']:
#     if sets == 'dict_alt_':
#         trg_dict = res_dicts_alt
#     else:
#         trg_dict = res_dicts
#     name_list = list(trg_dict.keys())
#     for name in name_list:
#         file_tail = sets + name + '.txt'
#         write_path = '../data/dict/pre/{}'.format(file_tail)
#         with open(write_path, 'w+') as f:
#             f.write('\n'.join(trg_dict[name])) 
#             f.close()
    

In [ ]:
import pandas as pd
human_test_path = '../data/weak_data/KIR/test/tag1.txt'

human_list = []
with open(human_test_path, 'r') as f:
    for i in f.readlines():
        human_list.append(i.strip().split())
    f.close()

In [ ]:
ham = ['O', 'O', 'O', 'O', 'O', 'B-Body-function-and-measurement', 'B-Medical-condition', 'O', 'B-Body-function-and-measurement', 'O', 'O', 'O']
idx = 0
d = {'origin':origin[idx], 'lemma':lemma[idx], 'human':human_list[idx], 'ham':ham}
pd.DataFrame(d)

In [ ]:
case[0]

In [ ]:
# _
# Person
# 

In [ ]:
# Tense=Pres
# Person=3
# Mood=Ind

In [ ]:
[i for i in range(len(pos[:thre])) if pos[:thre][i] == 'NOUN']

In [ ]:
pos[:thre][0]

In [ ]:
token_ = [j for i in origin[:thre] for j in i]
pos_ = [j for i in pos[:thre] for j in i]
human_ = [j for i in human_list[:thre] for j in i]
case_ = [j for i in case[:thre] for j in i]

In [ ]:
filter_idx = [i for i in range(len(pos_)) if pos_[i] == 'NOUN']
filter_idx_1 = [i for i in range(len(token_)) if pos_[i] == 'NOUN']

In [ ]:
from IPython.display import display
thre = 100
d = {'token': [token_[j] for j in filter_idx], 'pos': [pos_[j] for j in filter_idx], 'human': [human_[j] for j in filter_idx], 'case':[case_[j] for j in filter_idx]}
pd.set_option('display.max_rows', None)
df = pd.DataFrame(d)
df
o_case_list = []
noo_case_list = []
for idx, row in df.iterrows():
    if row['human'] != 'O':
        noo_case_list.append(row['case'])
    else:
        o_case_list.append(row['case'])

In [ ]:
set(o_case_list) - set(noo_case_list)

In [ ]:
df

In [ ]:
row['case']

In [ ]:
label2code = {}
code2label = {}
for k, v in mesh_dict.items():
    if k != '':
        label2code[v['prefLabel']] = k
        code2label[k] = v['prefLabel']


In [ ]:
name_dict = {}
for k, v in flat_dict.items():
    name_dict[code2label[k]] = [code2label[i] for i in list(v)]

In [ ]:
for k, v in name_dict.items():
    if 'vaikeutunut' in v:
        print(k)

In [ ]:
import json 
import re

In [ ]:
name_list = ['Anatomical-structure', 'Body-function-and-measurement', 'Medical-condition', 'Medical-device', 'Medical-procedure', 'Medication']

In [ ]:
res_dict = {}

for i in name_list:
    res = expand_item(res_dicts[i], res_dicts_alt[i])
    res_dict[i] = res
    
source = 'haavakontrolliin'
score_list = []

for k, v in res_dict.items():
    max_val = 0
    # max_idx = 0
    if len(source.split()) == 1:
        v_list = v[0]
    elif len(source.split()) == 2:
        v_list = v[1]
    else:
        v_list = v[2]
    for j in range(len(v_list)):
        item = v_list[j]
        fuzz_score = fuzz.ratio(source, item)
        if fuzz_score > max_val:
            max_val = fuzz_score
            # max_idx = j
    score_list.append(max_val)

max_score = np.max(score_list)
    # score_idx_list.append(max_idx)

In [ ]:
def beam_mapping(source_ele, r_dicts, r_dicts_alt, window, mask_list, check_thres):
  
  # print(source)
  labels = []
  # label_space =  list(r_dicts.keys())
  ratio_thres = 0.5
  # if ratio_score is too low --> subwording --> lose too much information
  
  # source_ele = source#.split(' ')
  source_len = len(source_ele)
  idx = 0
    
  new_source_ele = []
  new_mask_list = []
  mask_shift = 0
  for i in range(len(source_ele)):
        # print(source)
        source = source_ele[i]
        if '#' in source:
            source_list = source.split('#')
            # print(source_list)
            ratio_score = len(source_list[0])/len(''.join(source_list))
            # print(ratio_score)
            if ratio_score < ratio_thres:
                source = ''.join(source_list)
            else:
                source = source_list[0]
            # print(ratio_score)
            # print(source)
        if ' ' in source:
            new_source_ele += source.split()
            new_mask_list.append(mask_list[i-mask_shift])
            new_mask_list.append('F')
            mask_shift += 1
        else:
            new_source_ele.append(source)
            new_mask_list.append(mask_list[i-mask_shift])
        # new_source_ele.append(source)
  source_ele = new_source_ele
    # raise ValueError(1)
    
  name_list = ['Anatomical-structure', 'Body-function-and-measurement', 'Medical-condition', 'Medical-device', 'Medical-procedure', 'Medication']
        
  res_dict = {}

  for i in name_list:
    res = expand_item(r_dicts[i], r_dicts_alt[i])
    # res = expand_item_pool(r_dicts[i], r_dicts_alt[i])
    # print(res[0])
    # raise ValueError('1')
    # print(len(res[1]))
    # print(len(res[2]))
    # print('-'*10)
    res_dict[i] = res

  # raise ValueError('1')
  mask_list[0]
  while idx < source_len:
    if mask_list[idx] == 'T':
      if (source_len - idx) >= window:
        step = window
      else:
        step = source_len - idx
      candidate_idx = []
      candidate_value = []
      for shift in range(1, step+1):
        term = source_ele[idx:idx + shift]
        term_str = ' '.join(term)
        # !!! original !!!
        # _, candidate = candidates(term_str, r_dicts)
        # _, candidate_alt = candidates(term_str, r_dicts_alt)
        # s1 = cal_score(term_str, candidate)
        # s2 = cal_score(term_str, candidate_alt)
        # max_idx, max_value = label_assign(s1, s2)
        # !!! no accelerating !!!
        # source = term_str
        # score_list = []
        # for i in name_list:
        #     res = expand_item(r_dicts[i], r_dicts_alt[i])
        #     max_val = 0
        #     # max_idx = 0
        #     for j in range(len(res)):
        #         item = res[j]
        #         if fuzz.ratio(source, item) > max_val:
        #             max_val = fuzz.ratio(source, item)
        #             # max_idx = j
        #     score_list.append(max_val)
        # !!! accelerating
        source = term_str
        score_list = []

        for k, v in res_dict.items():
            max_val = 0
            # max_idx = 0
            # print(v)
            # raise ValueError(1)
            # fir_tok = source[0]
            # print(v[0])
            
            # if fir_tok in v[0]:
            # -----------------------
            # code below --> map tokens in phrase
            if len(source.split()) == 1:
                if source[0] in list(v[0].keys()):
                    v_list = v[0][source[0]]
                else:
                    v_list = set()
            elif len(source.split()) == 2:
                if source[0] in list(v[1].keys()):
                    v_list = v[1][source[0]]
                else:
                    v_list = set()
            elif len(source.split()) == 3:
                if source[0] in list(v[2].keys()):
                    v_list = v[2][source[0]]
                    # print(v_list)
                    # raise ValueError(1)
                else:
                    v_list = set()
            v_list_ = list(v_list)
            # -----------------------
            # code below --> pooling all phrases
            # v_list_ = 
            # for j in range(len(v_list_)):
            #     item = v_list_[j]
            #     item_list = item.split()
#             source_list = source.split()
#             # print(source)
#             # map 1 token; 2 token; 3 token
#             sum_max_val = 0
#             for z in range(len(source_list)):
#                 source_tok = source_list[z]
#                 if source_tok[0] in list(v.keys()):
#                     item_list = v[source_tok[0]]
#                 else:
#                     item_list = []
#                 # print(item_list)
#                 # raise ValueError(1)
#                 tmp_max_val = 0
#                 for item_tok in item_list:
#                     # print(source_list)
#                     # print(source_tok)
#                     # print(item_list)
#                     # raise ValueError(1)
#                     # for zz in 
#                     map_len = int(len(source_tok)*check_thres)
#                     # print(item)
#                     if len(item_tok) < map_len:
#                         break
#                     else:
#                         # if fuzz.ratio(source, item) > max_val:
#                         #     max_val = fuzz.ratio(source, item)
#                         if source_tok[:map_len] == item_tok[:map_len]:
#                             # print(source[:map_len])
#                             # print(item)
#                             # print(source_tok)
#                             # print(item_tok)
#                             # raise ValueError(1)
#                             IoU_score = map_len/len(item_tok)
#                             # print(IoU_score)
#                             # raise ValueError(1)
#                             if IoU_score > sum_max_val:
#                                 tmp_max_val = IoU_score
#                 sum_max_val += tmp_max_val
#             sum_max_val = sum_max_val#/len(source_list)
#             if sum_max_val > max_val:
#                 max_val = sum_max_val
                
#                 # fuzz_score = fuzz.ratio(source, item)
#                 # # print(source)
#                 # # print(item)
#                 # # raise ValueError('1')
#                 # if fuzz_score > max_val:
#                 #     max_val = fuzz_score
#                 #     # max_idx = j
#             # score_list.append(max_val/len(source_list))
#             score_list.append(max_val)
            # -------------------------
        # print(source)
        # print('_'*10)
        # print(score_list)
        # raise ValueError(1)
            # map_thres = 0.9
            # print(source)
            # map_len = int(len(source)*check_thres)
            # print(map_len)
            # print(source[:map_len])
            # print(len(source.split()))
            # raise ValueError(1)
            for j in range(len(v_list_)):
                item = v_list_[j]
                item_list = item.split()
                source_list = source.split()
                # map 1 token; 2 token; 3 token
                sum_max_val = 0
                # print(source_list)
                # print(item_list)
                # raise ValueError(1)
                for z in range(len(source_list)):
                    source_tok = source_list[z]
                    item_tok = item_list[z]
                    # try:
                    #     source_tok = source_list[z]
                    #     item_tok = item_list[z]
                    # except IndexError:
                    #     print(z)
                    #     print(source_list)
                    #     print(item_list)
                    map_len = int(len(source_tok)*check_thres)
                    # print(item)
                    if len(item_tok) < map_len:
                        break
                    else:
                        # if fuzz.ratio(source, item) > max_val:
                        #     max_val = fuzz.ratio(source, item)
                        # raise ValueError(1)
                        if source_tok[:map_len] == item_tok[:map_len]:
                            # print(source[:map_len])
                            # print(item)
                            IoU_score = map_len/len(item)
                            if IoU_score > sum_max_val:
                                sum_max_val = IoU_score
                                
                if sum_max_val > max_val:
                    max_val = sum_max_val
                
                # fuzz_score = fuzz.ratio(source, item)
                # # print(source)
                # # print(item)
                # # raise ValueError('1')
                # if fuzz_score > max_val:
                #     max_val = fuzz_score
                #     # max_idx = j
            score_list.append(max_val/len(source_list))
        # raise ValueError(1)
        # print(source)
        # print(source)
        # print(score_list)
        # print(term_str)
        # score_list = []
        # for i in label_space:
        #     res = expand_item(r_dicts[i], r_dicts_alt[i])
        #     max_val = 0
        #     # max_idx = 0
        #     for j in range(len(res)):
        #         item = res[j]
        #         if fuzz.ratio(term_str, item) > max_val:
        #             max_val = fuzz.ratio(source, item)
        #             # max_idx = j
        #     score_list.append(max_val)
        # print(score_list)
        # max_score = np.max(score_list)
        candidate_idx.append(np.argmax(score_list))
        candidate_value.append(np.max(score_list))
        
      # raise ValueError(1)
      if np.max(np.array(candidate_value)) > 0.5:
        # the position of best matching
        max_pos = np.argmax(np.array(candidate_value))
        # the label index of best matching
        label_idx = candidate_idx[max_pos]
        win_shift = max_pos + 1
        idx += win_shift
        label = name_list[label_idx]
        if win_shift == 1:
          labels.append('B-' + label)
        else:
          labels.append('B-' + label)
          for i in range(win_shift - 1):
            labels.append('I-' + label)
      else:
        labels.append('O')
        idx += 1
    else:
      labels.append('O')
      idx += 1

  return labels


In [ ]:
s_idx = 5223
a = beam_mapping(lemma[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx], 1.0)

In [ ]:
len(mask[s_idx])

In [ ]:
len(lemma[s_idx])

In [ ]:
import time

In [ ]:
if __name__ == '__main__':
    # obtain labeling dicts
    start_time = time.time()
    sets = 'KIR'
    input_path = '../data/lemma_data/{}/full_sample.conllu'.format(sets)
    mesh_path = '../tools/fin-mesh/mesh-skos.ttl'
    sent_path = '../data/processed_data/{}/full_sample_anno.txt'.format(sets)
    label_path = '../data/processed_data/{}/label.txt'.format(sets)

    mesh_dict = Mesh_dict(mesh_path)
    rel_list = Estab_rel(mesh_dict)
    window = 3
    # mesh:D055754 is the top one, also it is a English term -> remove
    res_dict, _ = Build_graph(rel_list)
    flat_dict = Flat_dict(res_dict)
    anno_dict = create_new_dicts(flat_dict, mesh_dict)
    # create res dict with label lakes
    # res_dicts, res_dicts_alt = create_res_dicts(anno_dict, mesh_dict)
    res_dicts, res_dicts_alt = reverse_dict()
    origin, lemma, mask, case, pos = lemma_tokens(input_path)
    # origin, lemma, mask = lemma_tokens(input_path)
    
    raise ValueError('1')

    labels_list = []
    token_list = []
    # print(len(origin))
    for s_idx in tqdm(range(len(origin))):
        #origin_tmp = ' '.join(origin[s_idx])
        #lemma_tmp = ' '.join(lemma[s_idx])
        labels_tmp = beam_mapping(lemma[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx], 1.0)
        # labels_tmp = beam_mapping(origin[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx])
        # print(labels_tmp)
        # raise ValueError(1)
        # print(' '.join(labels_tmp))
        # collection
        labels_list.append(labels_tmp)
        token_list.append(origin[s_idx])
        # if s_idx > 10000:
        #     break
    # write to the file
    write_list(label_path, labels_list)
    write_list(sent_path, token_list)
    end_time = time.time()
    with open(label_path.replace('label', 'time'), 'w+') as f:
        f.write(str(end_time - start_time))
        f.close()
    # print('Done')
  

In [ ]:
len(lemma[5223])

In [ ]:
len(mask[5223])

In [ ]:
len(origin[5223])

In [ ]:
origin[5223]

In [ ]:
len(mask[5223])

In [ ]:
if __name__ == '__main__':
    # obtain labeling dicts
    sets = 'RTG'
    input_path = '../data/lemma_data/{}/full_sample.conllu'.format(sets)
    mesh_path = '../tools/fin-mesh/mesh-skos.ttl'
    sent_path = '../data/processed_data/{}/full_sample_anno.txt'.format(sets)
    label_path = '../data/processed_data/{}/label.txt'.format(sets)

    mesh_dict = Mesh_dict(mesh_path)
    rel_list = Estab_rel(mesh_dict)
    window = 3
    # mesh:D055754 is the top one, also it is a English term -> remove
    res_dict, _ = Build_graph(rel_list)
    flat_dict = Flat_dict(res_dict)
    anno_dict = create_new_dicts(flat_dict, mesh_dict)
    # create res dict with label lakes
    # res_dicts, res_dicts_alt = create_res_dicts(anno_dict, mesh_dict)
    res_dicts, res_dicts_alt = reverse_dict()
    origin, lemma, mask, case, pos = lemma_tokens(input_path)
    # origin, lemma, mask = lemma_tokens(input_path)
    
    # raise ValueError('1')

    labels_list = []
    token_list = []
    # print(len(origin))
    for s_idx in tqdm(range(len(origin))):
        #origin_tmp = ' '.join(origin[s_idx])
        #lemma_tmp = ' '.join(lemma[s_idx])
        labels_tmp = beam_mapping(lemma[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx], 1.0)
        # labels_tmp = beam_mapping(origin[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx])
        # print(labels_tmp)
        # raise ValueError(1)
        # print(' '.join(labels_tmp))
        # collection
        labels_list.append(labels_tmp)
        token_list.append(origin[s_idx])
        # if s_idx > 10000:
        #     break
    # write to the file
    write_list(label_path, labels_list)
    write_list(sent_path, token_list)
    # print('Done')
  

In [ ]:
if __name__ == '__main__':
    # obtain labeling dicts
    sets = 'SAD'
    input_path = '../data/lemma_data/{}/full_sample.conllu'.format(sets)
    mesh_path = '../tools/fin-mesh/mesh-skos.ttl'
    sent_path = '../data/processed_data/{}/full_sample_anno.txt'.format(sets)
    label_path = '../data/processed_data/{}/label.txt'.format(sets)

    mesh_dict = Mesh_dict(mesh_path)
    rel_list = Estab_rel(mesh_dict)
    window = 3
    # mesh:D055754 is the top one, also it is a English term -> remove
    res_dict, _ = Build_graph(rel_list)
    flat_dict = Flat_dict(res_dict)
    anno_dict = create_new_dicts(flat_dict, mesh_dict)
    # create res dict with label lakes
    # res_dicts, res_dicts_alt = create_res_dicts(anno_dict, mesh_dict)
    res_dicts, res_dicts_alt = reverse_dict()
    origin, lemma, mask, case, pos = lemma_tokens(input_path)
    # origin, lemma, mask = lemma_tokens(input_path)
    
    # raise ValueError('1')

    labels_list = []
    token_list = []
    # print(len(origin))
    for s_idx in tqdm(range(len(origin))):
        #origin_tmp = ' '.join(origin[s_idx])
        #lemma_tmp = ' '.join(lemma[s_idx])
        labels_tmp = beam_mapping(lemma[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx], 1.0)
        # labels_tmp = beam_mapping(origin[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx])
        # print(labels_tmp)
        # raise ValueError(1)
        # print(' '.join(labels_tmp))
        # collection
        labels_list.append(labels_tmp)
        token_list.append(origin[s_idx])
        # if s_idx > 10000:
        #     break
    # write to the file
    write_list(label_path, labels_list)
    write_list(sent_path, token_list)
    # print('Done')
  

In [ ]:
if __name__ == '__main__':
    # obtain labeling dicts
    sets = 'LAH'
    input_path = '../data/lemma_data/{}/full_sample.conllu'.format(sets)
    mesh_path = '../tools/fin-mesh/mesh-skos.ttl'
    sent_path = '../data/processed_data/{}/full_sample_anno.txt'.format(sets)
    label_path = '../data/processed_data/{}/label.txt'.format(sets)

    mesh_dict = Mesh_dict(mesh_path)
    rel_list = Estab_rel(mesh_dict)
    window = 3
    # mesh:D055754 is the top one, also it is a English term -> remove
    res_dict, _ = Build_graph(rel_list)
    flat_dict = Flat_dict(res_dict)
    anno_dict = create_new_dicts(flat_dict, mesh_dict)
    # create res dict with label lakes
    # res_dicts, res_dicts_alt = create_res_dicts(anno_dict, mesh_dict)
    res_dicts, res_dicts_alt = reverse_dict()
    origin, lemma, mask, case, pos = lemma_tokens(input_path)
    # origin, lemma, mask = lemma_tokens(input_path)
    
    # raise ValueError('1')

    labels_list = []
    token_list = []
    # print(len(origin))
    for s_idx in tqdm(range(len(origin))):
        #origin_tmp = ' '.join(origin[s_idx])
        #lemma_tmp = ' '.join(lemma[s_idx])
        labels_tmp = beam_mapping(lemma[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx], 1.0)
        # labels_tmp = beam_mapping(origin[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx])
        # print(labels_tmp)
        # raise ValueError(1)
        # print(' '.join(labels_tmp))
        # collection
        labels_list.append(labels_tmp)
        token_list.append(origin[s_idx])
        # if s_idx > 10000:
        #     break
    # write to the file
    write_list(label_path, labels_list)
    write_list(sent_path, token_list)
    # print('Done')
  

In [ ]:
if __name__ == '__main__':
    # obtain labeling dicts
    sets = 'OPER'
    input_path = '../data/lemma_data/{}/full_sample.conllu'.format(sets)
    mesh_path = '../tools/fin-mesh/mesh-skos.ttl'
    sent_path = '../data/processed_data/{}/full_sample_anno.txt'.format(sets)
    label_path = '../data/processed_data/{}/label.txt'.format(sets)

    mesh_dict = Mesh_dict(mesh_path)
    rel_list = Estab_rel(mesh_dict)
    window = 3
    # mesh:D055754 is the top one, also it is a English term -> remove
    res_dict, _ = Build_graph(rel_list)
    flat_dict = Flat_dict(res_dict)
    anno_dict = create_new_dicts(flat_dict, mesh_dict)
    # create res dict with label lakes
    # res_dicts, res_dicts_alt = create_res_dicts(anno_dict, mesh_dict)
    res_dicts, res_dicts_alt = reverse_dict()
    origin, lemma, mask, case, pos = lemma_tokens(input_path)
    # origin, lemma, mask = lemma_tokens(input_path)
    
    # raise ValueError('1')

    labels_list = []
    token_list = []
    # print(len(origin))
    for s_idx in tqdm(range(len(origin))):
        #origin_tmp = ' '.join(origin[s_idx])
        #lemma_tmp = ' '.join(lemma[s_idx])
        labels_tmp = beam_mapping(lemma[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx], 1.0)
        # labels_tmp = beam_mapping(origin[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx])
        # print(labels_tmp)
        # raise ValueError(1)
        # print(' '.join(labels_tmp))
        # collection
        labels_list.append(labels_tmp)
        token_list.append(origin[s_idx])
        # if s_idx > 10000:
        #     break
    # write to the file
    write_list(label_path, labels_list)
    write_list(sent_path, token_list)
    # print('Done')
  

In [ ]:
# if __name__ == '__main__':
#     # obtain labeling dicts
#     sets = 'test'
#     input_path = '../data/lemma_data/{}/1.conllu'.format(sets)
#     mesh_path = '../tools/fin-mesh/mesh-skos.ttl'
#     sent_path = '../data/processed_data/{}/full_sample_anno.txt'.format(sets)
#     label_path = '../data/processed_data/{}/label.txt'.format(sets)

#     mesh_dict = Mesh_dict(mesh_path)
#     rel_list = Estab_rel(mesh_dict)
#     window = 3
#     # mesh:D055754 is the top one, also it is a English term -> remove
#     res_dict, _ = Build_graph(rel_list)
#     flat_dict = Flat_dict(res_dict)
#     anno_dict = create_new_dicts(flat_dict, mesh_dict)
#     # create res dict with label lakes
#     res_dicts, res_dicts_alt = create_res_dicts(anno_dict, mesh_dict)
#     origin, lemma, mask = lemma_tokens(input_path)

#     labels_list = []
#     token_list = []
#     print(len(origin))
#     for s_idx in range(len(origin)):
#         #origin_tmp = ' '.join(origin[s_idx])
#         #lemma_tmp = ' '.join(lemma[s_idx])
#         labels_tmp = beam_mapping(lemma[s_idx], res_dicts, res_dicts_alt, window, mask[s_idx])
#         # collection
#         labels_list.append(labels_tmp)
#         token_list.append(origin[s_idx])
#         # if s_idx > 10000:
#         #     break
#     # write to the file
#     write_list(label_path, labels_list)
#     write_list(sent_path, token_list)
  